In [1]:
import pandas as pd


Import and clean GBG data

In [2]:
df_gbg = pd.read_csv('../Data/smhi_temp_data_gbg.csv', usecols=[0, 1, 2, 3], header=11, sep=';')

In [3]:
df_gbg

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,1961-01-01,06:00:00,0.8,G
1,1961-01-01,12:00:00,1.0,G
2,1961-01-01,18:00:00,1.4,G
3,1961-01-02,06:00:00,1.8,G
4,1961-01-02,12:00:00,2.2,G
...,...,...,...,...
308145,2023-07-01,02:00:00,16.1,G
308146,2023-07-01,03:00:00,16.1,G
308147,2023-07-01,04:00:00,16.2,G
308148,2023-07-01,05:00:00,16.3,G


In [4]:
df_gbg['Snittemperatur'] = df_gbg.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)
df_gbg = df_gbg.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()


In [5]:
df_gbg.dtypes

Datum              object
Tid (UTC)          object
Kvalitet           object
Snittemperatur    float64
dtype: object

In [6]:
df_gbg['Datum'] = pd.to_datetime(df_gbg['Datum'])
df_gbg['Year'] = df_gbg['Datum'].dt.year
df_gbg['Month'] = df_gbg['Datum'].dt.month
df_gbg = df_gbg.drop(columns=['Tid (UTC)'])

Import and clean Säve data

In [7]:
df_save = pd.read_csv('../Data/smhi_temp_data_save.csv', usecols=[0, 1, 2, 3], header=6, sep=';')

In [8]:
df_save['Snittemperatur'] = df_save.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)
df_save = df_save.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [9]:
df_save['Datum'] = pd.to_datetime(df_save['Datum'])
df_save['Year'] = df_save['Datum'].dt.year
df_save['Month'] = df_save['Datum'].dt.month
df_save = df_save.drop(columns=['Tid (UTC)'])

In [10]:
df_save

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1944-01-01,G,3.20,1944,1
1,1944-01-02,G,3.60,1944,1
2,1944-01-03,G,2.80,1944,1
3,1944-01-04,G,-4.47,1944,1
4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...
22980,2006-12-01,Y,8.03,2006,12
22981,2006-12-02,Y,8.81,2006,12
22982,2006-12-03,Y,8.11,2006,12
22983,2006-12-04,Y,8.57,2006,12


In [11]:
df_gbg

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1961-01-01,G,1.07,1961,1
1,1961-01-02,G,2.27,1961,1
2,1961-01-03,G,2.27,1961,1
3,1961-01-04,G,2.53,1961,1
4,1961-01-05,G,2.13,1961,1
...,...,...,...,...,...
19909,2023-06-27,G,18.18,2023,6
19910,2023-06-28,G,19.72,2023,6
19911,2023-06-29,G,19.22,2023,6
19912,2023-06-30,G,17.40,2023,6


Save both datasets as csv and pkl

In [12]:
# df_gbg.to_csv('../data/prepared_data/daily_temp_gbg.csv', index=False, mode='w')
# df_save.to_csv('../data/prepared_data/daily_temp_save.csv', index=False, mode='w')

In [13]:
# df_gbg.to_pickle('../Dataframes/df_daily_temp_gbg.pkl')
# df_save.to_pickle('../Dataframes/df_daily_temp_save.pkl')

Combine borh GBG and Säve
Let Säve data run all the way until last day, let GBG take over

In [14]:
# Last fay of Säve data was not a complete day, remove it.
df_save.drop(df_save.tail(1).index, inplace=True)


In [15]:
df_save

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1944-01-01,G,3.20,1944,1
1,1944-01-02,G,3.60,1944,1
2,1944-01-03,G,2.80,1944,1
3,1944-01-04,G,-4.47,1944,1
4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...
22979,2006-11-30,Y,9.98,2006,11
22980,2006-12-01,Y,8.03,2006,12
22981,2006-12-02,Y,8.81,2006,12
22982,2006-12-03,Y,8.11,2006,12


In [16]:
last_day_save = df_save['Datum'][-1:]

In [17]:
last_day_save = str(last_day_save.values[0])[0:10]

In [18]:
last_day_save

'2006-12-04'

In [19]:
df_gbg_compile = df_gbg[df_gbg['Datum'] > last_day_save]

In [20]:
df_compiled = pd.concat([df_save, df_gbg_compile]).reset_index()

In [21]:
df_compiled

,index,Datum,Kvalitet,Snittemperatur,Year,Month
0,0,1944-01-01,G,3.20,1944,1
1,1,1944-01-02,G,3.60,1944,1
2,2,1944-01-03,G,2.80,1944,1
3,3,1944-01-04,G,-4.47,1944,1
4,4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...,...
29002,19909,2023-06-27,G,18.18,2023,6
29003,19910,2023-06-28,G,19.72,2023,6
29004,19911,2023-06-29,G,19.22,2023,6
29005,19912,2023-06-30,G,17.40,2023,6


In [22]:
# df_compiled.to_csv('../data/prepared_data/compiled_daily_temp_gbg_save.csv', index=False)

In [23]:
# df_compiled.to_pickle('../Dataframes/df_compiled_daily_temp_gbg_save.pkl')

Make monthly temp data

In [24]:
df_gbg_monthly_temp = df_gbg.groupby(['Year', 'Month'])['Snittemperatur'].mean().round(2).reset_index()

In [56]:
df_gbg_monthly_temp

,Year,Month,Snittemperatur
0,1961,1,-0.25
1,1961,2,2.82
2,1961,3,5.20
3,1961,4,8.78
4,1961,5,11.96
...,...,...,...
655,2023,3,2.39
656,2023,4,8.06
657,2023,5,13.52
658,2023,6,18.59


In [25]:
df_save_monthly_temp = df_save.groupby(['Year', 'Month'])['Snittemperatur'].mean().round(2).reset_index()

In [55]:
df_save_monthly_temp

,Year,Month,Snittemperatur
0,1944,1,2.03
1,1944,2,0.18
2,1944,3,0.96
3,1944,4,6.08
4,1944,5,10.74
...,...,...,...
751,2006,8,16.95
752,2006,9,15.93
753,2006,10,11.17
754,2006,11,6.99


In [26]:
# df_gbg_monthly_temp.to_csv('../data/prepared_data/monthly_temp_gbg.csv', index=False)
# df_save_monthly_temp.to_csv('../data/prepared_data/monthly_temp_save.csv', index=False)

In [27]:
# df_gbg_monthly_temp.to_pickle('../Dataframes/df_monthly_temp_gbg.pkl')
# df_save_monthly_temp.to_pickle('../Dataframes/df_monthly_temp_save.pkl')

Combine the two

In [67]:
# Limit the Säve datset to not include last month, as it is not complete

df_save_monthly_limited = df_save_monthly_temp.drop(df_save_monthly_temp.index[-1])
save_last_year = df_save_monthly_limited['Year'][-1:].values[0]
save_last_month = df_save_monthly_limited['Month'][-1:].values[0]

In [84]:
# Limit the gbg dataset to only include dates from dec 2006

df_gbg_monthly_limited = df_gbg_monthly_temp[(df_gbg_monthly_temp['Year'] > save_last_year) | ((df_gbg_monthly_temp['Year'] == save_last_year) & (df_gbg_monthly_temp['Month'] == save_last_month))]

In [85]:
df_gbg_monthly_limited

,Year,Month,Snittemperatur
459,2006,11,7.65
461,2007,1,3.94
462,2007,2,0.60
463,2007,3,6.03
464,2007,4,9.10
...,...,...,...
655,2023,3,2.39
656,2023,4,8.06
657,2023,5,13.52
658,2023,6,18.59


In [98]:
df_compiled_monthly_temp = pd.concat([df_save_monthly_limited, df_gbg_monthly_limited]).reset_index().drop(columns=('index'))

In [100]:
df_compiled_monthly_temp

,Year,Month,Snittemperatur
0,1944,1,2.03
1,1944,2,0.18
2,1944,3,0.96
3,1944,4,6.08
4,1944,5,10.74
...,...,...,...
950,2023,3,2.39
951,2023,4,8.06
952,2023,5,13.52
953,2023,6,18.59


In [101]:
# for i in range(1944, 2023):
#     for ind, val in df_compiled_monthly_temp.iterrows():
#         if i == val['Year']:    
#             year = val['Year']
#             print(f'{year} // {ind}')
#             break

1944.0 // 0
1945.0 // 12
1946.0 // 24
1947.0 // 36
1948.0 // 48
1949.0 // 60
1950.0 // 72
1951.0 // 84
1952.0 // 96
1953.0 // 108
1954.0 // 120
1955.0 // 132
1956.0 // 144
1957.0 // 156
1958.0 // 168
1959.0 // 180
1960.0 // 192
1961.0 // 204
1962.0 // 216
1963.0 // 228
1964.0 // 240
1965.0 // 252
1966.0 // 264
1967.0 // 276
1968.0 // 288
1969.0 // 300
1970.0 // 312
1971.0 // 324
1972.0 // 336
1973.0 // 348
1974.0 // 360
1975.0 // 372
1976.0 // 384
1977.0 // 396
1978.0 // 408
1979.0 // 420
1980.0 // 432
1981.0 // 444
1982.0 // 456
1983.0 // 468
1984.0 // 480
1985.0 // 492
1986.0 // 504
1987.0 // 516
1988.0 // 528
1989.0 // 540
1990.0 // 552
1991.0 // 564
1992.0 // 576
1993.0 // 588
1994.0 // 600
1995.0 // 612
1996.0 // 624
1997.0 // 636
1998.0 // 648
1999.0 // 660
2000.0 // 672
2001.0 // 684
2002.0 // 696
2003.0 // 708
2004.0 // 720
2005.0 // 732
2006.0 // 744
2007.0 // 756
2008.0 // 768
2009.0 // 780
2010.0 // 792
2011.0 // 804
2012.0 // 816
2013.0 // 828
2014.0 // 840
2015.0 // 852
20

In [102]:
# df_compiled_monthly_temp.to_csv('../data/prepared_data/compiled_monthly_temp_gbg_save.csv', index=False, mode='w')
# df_compiled_monthly_temp.to_pickle('../Dataframes/df_compiled_monthly_temp_gbg_save.pkl')